In [1]:
#Importdependancy.
import pandas as pd
import numpy as np
import os
import csv
import warnings
warnings.filterwarnings('ignore')
from io import StringIO
import zipfile
from pathlib import Path
import sqlalchemy as db
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, func
from sqlalchemy import select
from sqlalchemy import create_engine
from requests.api import head
from sqlalchemy.dialects.mysql import insert


In [2]:
#Read dataframe.
demographic_ml_df=pd.read_csv("Resources/demographic_ml_df.csv",header=0)
demographic_ml_df

,Unnamed: 0,year,country_origin,country_asylum,share_borders,female_0to4,female_5to11,female_12to17,female_18to59,female_60,...,unknown_demographic,male_60,male_other,Male total,total,recognized_decisions,complementary_protection,rejected,otherwise_closed,total_decisions
0,0,2001,Afghanistan,Australia,0,0,0,0,0,0,...,6262,0,0,0,6262,1914.0,0.0,522.0,10.0,2446.0
1,1,2001,Afghanistan,Austria,0,0,0,0,0,0,...,1049,0,0,0,1049,432.0,0.0,335.0,0.0,767.0
2,2,2001,Afghanistan,Azerbaijan,0,8,12,10,38,0,...,0,0,0,175,243,226.0,0.0,45.0,0.0,271.0
3,3,2001,Afghanistan,Belarus,0,9,40,36,81,6,...,0,5,0,282,454,85.0,0.0,25.0,51.0,161.0
4,4,2001,Afghanistan,Belgium,0,0,0,0,0,0,...,0,0,213,213,357,100.0,0.0,42.0,0.0,142.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61194,61194,2021,Zimbabwe,Sweden,0,0,0,0,0,0,...,16,0,0,0,16,0.0,0.0,16.0,0.0,16.0
61195,61195,2021,Zimbabwe,Switzerland,0,0,9,0,0,0,...,0,0,0,0,9,0.0,0.0,5.0,0.0,5.0
61196,61196,2021,Zimbabwe,Thailand,0,0,0,0,0,0,...,0,0,0,9,9,NaN,NaN,NaN,NaN,NaN
61197,61197,2021,Zimbabwe,United Kingdom of Great Britain and Northern I...,0,0,0,0,0,0,...,1174,0,0,0,1174,44.0,5.0,53.0,10.0,112.0


In [3]:
#Renaming Columns.
demographic_ml_df=demographic_ml_df.rename(columns={"Female total":"female_total","Male total":"male_total","total":"total_gender"})
demographic_ml_df

,Unnamed: 0,year,country_origin,country_asylum,share_borders,female_0to4,female_5to11,female_12to17,female_18to59,female_60,...,unknown_demographic,male_60,male_other,male_total,total_gender,recognized_decisions,complementary_protection,rejected,otherwise_closed,total_decisions
0,0,2001,Afghanistan,Australia,0,0,0,0,0,0,...,6262,0,0,0,6262,1914.0,0.0,522.0,10.0,2446.0
1,1,2001,Afghanistan,Austria,0,0,0,0,0,0,...,1049,0,0,0,1049,432.0,0.0,335.0,0.0,767.0
2,2,2001,Afghanistan,Azerbaijan,0,8,12,10,38,0,...,0,0,0,175,243,226.0,0.0,45.0,0.0,271.0
3,3,2001,Afghanistan,Belarus,0,9,40,36,81,6,...,0,5,0,282,454,85.0,0.0,25.0,51.0,161.0
4,4,2001,Afghanistan,Belgium,0,0,0,0,0,0,...,0,0,213,213,357,100.0,0.0,42.0,0.0,142.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61194,61194,2021,Zimbabwe,Sweden,0,0,0,0,0,0,...,16,0,0,0,16,0.0,0.0,16.0,0.0,16.0
61195,61195,2021,Zimbabwe,Switzerland,0,0,9,0,0,0,...,0,0,0,0,9,0.0,0.0,5.0,0.0,5.0
61196,61196,2021,Zimbabwe,Thailand,0,0,0,0,0,0,...,0,0,0,9,9,NaN,NaN,NaN,NaN,NaN
61197,61197,2021,Zimbabwe,United Kingdom of Great Britain and Northern I...,0,0,0,0,0,0,...,1174,0,0,0,1174,44.0,5.0,53.0,10.0,112.0


In [4]:
#Dropp Unnamed:0 column.
demographic_ml_df= demographic_ml_df.drop('Unnamed: 0', axis=1)
demographic_ml_df

,year,country_origin,country_asylum,share_borders,female_0to4,female_5to11,female_12to17,female_18to59,female_60,female_other,...,unknown_demographic,male_60,male_other,male_total,total_gender,recognized_decisions,complementary_protection,rejected,otherwise_closed,total_decisions
0,2001,Afghanistan,Australia,0,0,0,0,0,0,0,...,6262,0,0,0,6262,1914.0,0.0,522.0,10.0,2446.0
1,2001,Afghanistan,Austria,0,0,0,0,0,0,0,...,1049,0,0,0,1049,432.0,0.0,335.0,0.0,767.0
2,2001,Afghanistan,Azerbaijan,0,8,12,10,38,0,0,...,0,0,0,175,243,226.0,0.0,45.0,0.0,271.0
3,2001,Afghanistan,Belarus,0,9,40,36,81,6,0,...,0,5,0,282,454,85.0,0.0,25.0,51.0,161.0
4,2001,Afghanistan,Belgium,0,0,0,0,0,0,144,...,0,0,213,213,357,100.0,0.0,42.0,0.0,142.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61194,2021,Zimbabwe,Sweden,0,0,0,0,0,0,0,...,16,0,0,0,16,0.0,0.0,16.0,0.0,16.0
61195,2021,Zimbabwe,Switzerland,0,0,9,0,0,0,0,...,0,0,0,0,9,0.0,0.0,5.0,0.0,5.0
61196,2021,Zimbabwe,Thailand,0,0,0,0,0,0,0,...,0,0,0,9,9,NaN,NaN,NaN,NaN,NaN
61197,2021,Zimbabwe,United Kingdom of Great Britain and Northern I...,0,0,0,0,0,0,0,...,1174,0,0,0,1174,44.0,5.0,53.0,10.0,112.0


In [5]:
#Check dropping null values.
demographic_ml_df =demographic_ml_df.dropna()
demographic_ml_df

,year,country_origin,country_asylum,share_borders,female_0to4,female_5to11,female_12to17,female_18to59,female_60,female_other,...,unknown_demographic,male_60,male_other,male_total,total_gender,recognized_decisions,complementary_protection,rejected,otherwise_closed,total_decisions
0,2001,Afghanistan,Australia,0,0,0,0,0,0,0,...,6262,0,0,0,6262,1914.0,0.0,522.0,10.0,2446.0
1,2001,Afghanistan,Austria,0,0,0,0,0,0,0,...,1049,0,0,0,1049,432.0,0.0,335.0,0.0,767.0
2,2001,Afghanistan,Azerbaijan,0,8,12,10,38,0,0,...,0,0,0,175,243,226.0,0.0,45.0,0.0,271.0
3,2001,Afghanistan,Belarus,0,9,40,36,81,6,0,...,0,5,0,282,454,85.0,0.0,25.0,51.0,161.0
4,2001,Afghanistan,Belgium,0,0,0,0,0,0,144,...,0,0,213,213,357,100.0,0.0,42.0,0.0,142.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61193,2021,Zimbabwe,South Africa,1,0,43,146,1481,73,0,...,0,70,0,2222,3965,13.0,0.0,737.0,207.0,957.0
61194,2021,Zimbabwe,Sweden,0,0,0,0,0,0,0,...,16,0,0,0,16,0.0,0.0,16.0,0.0,16.0
61195,2021,Zimbabwe,Switzerland,0,0,9,0,0,0,0,...,0,0,0,0,9,0.0,0.0,5.0,0.0,5.0
61197,2021,Zimbabwe,United Kingdom of Great Britain and Northern I...,0,0,0,0,0,0,0,...,1174,0,0,0,1174,44.0,5.0,53.0,10.0,112.0


In [6]:
demographic_ml_df.dtypes

year                          int64
country_origin               object
country_asylum               object
share_borders                 int64
female_0to4                   int64
female_5to11                  int64
female_12to17                 int64
female_18to59                 int64
female_60                     int64
female_other                  int64
female_total                  int64
male_0to4                     int64
male_5to11                    int64
male_12to17                   int64
male_18to59                   int64
unknown_demographic           int64
male_60                       int64
male_other                    int64
male_total                    int64
total_gender                  int64
recognized_decisions        float64
complementary_protection    float64
rejected                    float64
otherwise_closed            float64
total_decisions             float64
dtype: object

In [7]:
#Read dataframe.
event_ml_df=pd.read_csv("Resources/event_ml_df.csv",header=0)
event_ml_df

,Unnamed: 0,year,country_origin,disaster_deaths,disaster_affected,conflict_deaths
0,0,2001,Afghanistan,331,200270,864
1,1,2001,Algeria,921,45423,32
2,2,2001,Angola,48,39928,1038
3,3,2001,Azerbaijan,0,0,0
4,4,2001,Bangladesh,232,530150,6
...,...,...,...,...,...,...
1092,1092,2021,Turkey,9,561088,0
1093,1093,2021,Uganda,0,0,7
1094,1094,2021,Ukraine,1,1700,18
1095,1095,2021,Venezuela,0,0,19


In [8]:
#Dropp Unnamed:0 column.
event_ml_df= event_ml_df.drop('Unnamed: 0', axis=1)
event_ml_df

,year,country_origin,disaster_deaths,disaster_affected,conflict_deaths
0,2001,Afghanistan,331,200270,864
1,2001,Algeria,921,45423,32
2,2001,Angola,48,39928,1038
3,2001,Azerbaijan,0,0,0
4,2001,Bangladesh,232,530150,6
...,...,...,...,...,...
1092,2021,Turkey,9,561088,0
1093,2021,Uganda,0,0,7
1094,2021,Ukraine,1,1700,18
1095,2021,Venezuela,0,0,19


In [9]:
event_ml_df.dtypes


year                  int64
country_origin       object
disaster_deaths       int64
disaster_affected     int64
conflict_deaths       int64
dtype: object

In [10]:
#Read dataframe.
cleaned_conflict=pd.read_csv("Resources/cleaned conflict.csv",header=0)
cleaned_conflict

,year,conflict_name,dyad_name,where_coordinates,where_description,latitude,longitude,country,region,deaths_civilians
0,2017,Iraq: Government,Government of Iraq - IS,Kabul city,Iraqi embassy in Kabul,34.531094,69.162796,Afghanistan,Asia,6
1,2021,Iraq: Government,Government of Iraq - IS,Kabul international airport,Kabul airport (Abbey gate entrance),34.564444,69.217222,Afghanistan,Asia,183
2,2021,Iraq: Government,Government of Iraq - IS,Jalalabad town,Police District 7 of Jalalabad city,34.428844,70.455750,Afghanistan,Asia,2
3,2021,Iraq: Government,Government of Iraq - IS,Kabul city,Kabul city (district 15),34.531094,69.162796,Afghanistan,Asia,10
4,2002,Afghanistan: Government,Government of Afghanistan - Hizb-i Islami-yi A...,Kabul city,Kabul city,34.531094,69.162796,Afghanistan,Asia,30
...,...,...,...,...,...,...,...,...,...,...
197328,2019,Government of Zimbabwe (Rhodesia) - Civilians,Government of Zimbabwe (Rhodesia) - Civilians,Harare town,"Warren park, Harare city",-17.817777,31.044722,Zimbabwe (Rhodesia),Africa,1
197329,2019,Government of Zimbabwe (Rhodesia) - Civilians,Government of Zimbabwe (Rhodesia) - Civilians,Mutare town,Mutare,-18.975973,32.650092,Zimbabwe (Rhodesia),Africa,1
197330,2019,Government of Zimbabwe (Rhodesia) - Civilians,Government of Zimbabwe (Rhodesia) - Civilians,Harare town,Harare Central Business District,-17.817777,31.044722,Zimbabwe (Rhodesia),Africa,1
197331,2019,Government of Zimbabwe (Rhodesia) - Civilians,Government of Zimbabwe (Rhodesia) - Civilians,Mwenezi district,Mwenezi,-21.358380,30.706680,Zimbabwe (Rhodesia),Africa,2


In [11]:
cleaned_conflict.dtypes

year                   int64
conflict_name         object
dyad_name             object
where_coordinates     object
where_description     object
latitude             float64
longitude            float64
country               object
region                object
deaths_civilians       int64
dtype: object

In [12]:
#Renaming Columns.
cleaned_conflict=cleaned_conflict.rename(columns={'Year':'year','Conflict name':'conflict_name',\
                                                    'Dyad name':'dyad_name','Where coordinates':'where_coordinates','Where description':'where_description',\
                                                     'Latitude':'latitude','Longitude':'longitude','Country of origin':'country_origin','Region':'region','Deaths civilians':'best'})
cleaned_conflict

,year,conflict_name,dyad_name,where_coordinates,where_description,latitude,longitude,country,region,deaths_civilians
0,2017,Iraq: Government,Government of Iraq - IS,Kabul city,Iraqi embassy in Kabul,34.531094,69.162796,Afghanistan,Asia,6
1,2021,Iraq: Government,Government of Iraq - IS,Kabul international airport,Kabul airport (Abbey gate entrance),34.564444,69.217222,Afghanistan,Asia,183
2,2021,Iraq: Government,Government of Iraq - IS,Jalalabad town,Police District 7 of Jalalabad city,34.428844,70.455750,Afghanistan,Asia,2
3,2021,Iraq: Government,Government of Iraq - IS,Kabul city,Kabul city (district 15),34.531094,69.162796,Afghanistan,Asia,10
4,2002,Afghanistan: Government,Government of Afghanistan - Hizb-i Islami-yi A...,Kabul city,Kabul city,34.531094,69.162796,Afghanistan,Asia,30
...,...,...,...,...,...,...,...,...,...,...
197328,2019,Government of Zimbabwe (Rhodesia) - Civilians,Government of Zimbabwe (Rhodesia) - Civilians,Harare town,"Warren park, Harare city",-17.817777,31.044722,Zimbabwe (Rhodesia),Africa,1
197329,2019,Government of Zimbabwe (Rhodesia) - Civilians,Government of Zimbabwe (Rhodesia) - Civilians,Mutare town,Mutare,-18.975973,32.650092,Zimbabwe (Rhodesia),Africa,1
197330,2019,Government of Zimbabwe (Rhodesia) - Civilians,Government of Zimbabwe (Rhodesia) - Civilians,Harare town,Harare Central Business District,-17.817777,31.044722,Zimbabwe (Rhodesia),Africa,1
197331,2019,Government of Zimbabwe (Rhodesia) - Civilians,Government of Zimbabwe (Rhodesia) - Civilians,Mwenezi district,Mwenezi,-21.358380,30.706680,Zimbabwe (Rhodesia),Africa,2


In [13]:
cleaned_conflict.columns

Index(['year', 'conflict_name', 'dyad_name', 'where_coordinates',
       'where_description', 'latitude', 'longitude', 'country', 'region',
       'deaths_civilians'],
      dtype='object')

In [14]:
cleaned_conflict.dtypes

year                   int64
conflict_name         object
dyad_name             object
where_coordinates     object
where_description     object
latitude             float64
longitude            float64
country               object
region                object
deaths_civilians       int64
dtype: object

In [15]:
#Read dataframe.
cleaned_disasters=pd.read_csv("Resources/cleaned disasters.csv",header=0)
cleaned_disasters

,year,disaster_group,disaster_subgroup,disaster_type,disaster_subtype,country_origin,country_origin(ISO),region,continent,location,total_deaths,total_affected,geo_locations
0,2001,Natural,Meteorological,Extreme temperature,Cold wave,Afghanistan,AFG,Southern Asia,Asia,"Hirat, Faryab, Jawzjan, Balkh, Samangan, Sar-e...",150.0,100000.0,"Baghlan, Balkh, Faryab, Hirat, Jawzjan, Kunduz..."
1,2001,Natural,Hydrological,Flood,Riverine flood,Bolivia (Plurinational State of),BOL,South America,Americas,"Beni, Chuquisaca, Cochabamba, La Paz, Oruro, P...",41.0,357250.0,"Beni, Chuquisaca, Cochabamba, La Paz, Oruro, P..."
2,2001,Natural,Meteorological,Storm,Convective storm,China,CHN,Eastern Asia,Asia,"Xilin Gol, Xing'an, Hulunbuir, Ulaan Chab, Chi...",49.0,2574871.0,"Chifeng, Hulunbuir, Tongliao, Ulaan Chab, Xili..."
3,2001,Natural,Geophysical,Earthquake,Ground movement,China,CHN,Eastern Asia,Asia,"Yajiang Xian, Kangding Xian areas (Garze Tibet...",3.0,300109.0,Garzê Tibetan (Adm2).
4,2001,Natural,Hydrological,Flood,Flash flood,Indonesia,IDN,South-Eastern Asia,Asia,"Bogor district (Jawa Barat province), Bojonego...",130.0,80000.0,"Dki Jakarta (Adm1). Bogor, Bojonegoro, Jember,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3640,2021,Natural,Climatological,Wildfire,Forest fire,United States of America,USA,Northern America,Americas,Plumas County (California),3.0,1261.0,Plumas (Adm2).
3641,2021,Natural,Hydrological,Landslide,Mudslide,Uzbekistan,UZB,Central Asia,Asia,"Ferghana valley, Namangan Region",8.0,6.0,"Fergana, Namangan (Adm1)."
3642,2021,Natural,Hydrological,Flood,Flash flood,Viet Nam,VNM,South-Eastern Asia,Asia,"Minh Luong Commune, Viet Tien (Van Ban Distric...",3.0,180.0,"Ha Giang (Adm1). Tran Yen, Van Ban (Adm2)."
3643,2021,Natural,Hydrological,Flood,Flash flood,Yemen,YEM,Western Asia,Asia,"Sanaa, Ibb, Shabwa, Hodeida, Aden, Abyan, Al D...",13.0,22380.0,"Abyan, Aden, Al Dhale'e, Al Hudaydah, Hadramau..."


In [16]:
cleaned_disasters.columns

Index(['year', 'disaster_group', 'disaster_subgroup', 'disaster_type',
       'disaster_subtype', 'country_origin', 'country_origin(ISO)', 'region',
       'continent', 'location', 'total_deaths', 'total_affected',
       'geo_locations'],
      dtype='object')

In [17]:
#Renaming Columns.
cleaned_disasters=cleaned_disasters.rename(columns={'Year':'year','Disaster group':'disaster_group','Disaster subgroup':'disaster_subgroup',\
                                                    'Disaster type':'disaster_type','Disaster subtype':'disaster_subtype','Country of origin':'country_origin','Country of origin(ISO)':'country_origin (ISO)',\
                                                   'Region':'region','Continent':'continent','Location':'location','Total deaths':'total_deaths','Total affected':'total_affected','Geo locations':'geo_locations'})
cleaned_disasters


,year,disaster_group,disaster_subgroup,disaster_type,disaster_subtype,country_origin,country_origin(ISO),region,continent,location,total_deaths,total_affected,geo_locations
0,2001,Natural,Meteorological,Extreme temperature,Cold wave,Afghanistan,AFG,Southern Asia,Asia,"Hirat, Faryab, Jawzjan, Balkh, Samangan, Sar-e...",150.0,100000.0,"Baghlan, Balkh, Faryab, Hirat, Jawzjan, Kunduz..."
1,2001,Natural,Hydrological,Flood,Riverine flood,Bolivia (Plurinational State of),BOL,South America,Americas,"Beni, Chuquisaca, Cochabamba, La Paz, Oruro, P...",41.0,357250.0,"Beni, Chuquisaca, Cochabamba, La Paz, Oruro, P..."
2,2001,Natural,Meteorological,Storm,Convective storm,China,CHN,Eastern Asia,Asia,"Xilin Gol, Xing'an, Hulunbuir, Ulaan Chab, Chi...",49.0,2574871.0,"Chifeng, Hulunbuir, Tongliao, Ulaan Chab, Xili..."
3,2001,Natural,Geophysical,Earthquake,Ground movement,China,CHN,Eastern Asia,Asia,"Yajiang Xian, Kangding Xian areas (Garze Tibet...",3.0,300109.0,Garzê Tibetan (Adm2).
4,2001,Natural,Hydrological,Flood,Flash flood,Indonesia,IDN,South-Eastern Asia,Asia,"Bogor district (Jawa Barat province), Bojonego...",130.0,80000.0,"Dki Jakarta (Adm1). Bogor, Bojonegoro, Jember,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3640,2021,Natural,Climatological,Wildfire,Forest fire,United States of America,USA,Northern America,Americas,Plumas County (California),3.0,1261.0,Plumas (Adm2).
3641,2021,Natural,Hydrological,Landslide,Mudslide,Uzbekistan,UZB,Central Asia,Asia,"Ferghana valley, Namangan Region",8.0,6.0,"Fergana, Namangan (Adm1)."
3642,2021,Natural,Hydrological,Flood,Flash flood,Viet Nam,VNM,South-Eastern Asia,Asia,"Minh Luong Commune, Viet Tien (Van Ban Distric...",3.0,180.0,"Ha Giang (Adm1). Tran Yen, Van Ban (Adm2)."
3643,2021,Natural,Hydrological,Flood,Flash flood,Yemen,YEM,Western Asia,Asia,"Sanaa, Ibb, Shabwa, Hodeida, Aden, Abyan, Al D...",13.0,22380.0,"Abyan, Aden, Al Dhale'e, Al Hudaydah, Hadramau..."


In [18]:
cleaned_disasters.columns

Index(['year', 'disaster_group', 'disaster_subgroup', 'disaster_type',
       'disaster_subtype', 'country_origin', 'country_origin(ISO)', 'region',
       'continent', 'location', 'total_deaths', 'total_affected',
       'geo_locations'],
      dtype='object')

In [19]:
cleaned_disasters.dtypes

year                     int64
disaster_group          object
disaster_subgroup       object
disaster_type           object
disaster_subtype        object
country_origin          object
country_origin(ISO)     object
region                  object
continent               object
location                object
total_deaths           float64
total_affected         float64
geo_locations           object
dtype: object

In [20]:
#create engine.
engine = create_engine('sqlite:///DataBase/Refugee_migration.db', echo=False)

In [21]:
from sqlalchemy.exc import IntegrityError

try:
    demographic_ml_df.to_sql(name='demographic_ml', con=engine, if_exists='replace', index=True)
    event_ml_df.to_sql(name='event_ml', con=engine, if_exists='replace', index=True)
    cleaned_conflict.to_sql(name=' cleaned_conflict', con=engine, if_exists='replace', index=True)
    cleaned_disasters.to_sql(name=' cleaned_disasters', con=engine, if_exists='replace', index=True)
    
    # engine.execute(mytable.insert(), **kwargs)
except IntegrityError:
    pass